In [1]:
import os

print("🔍 Checking ALL Kaggle inputs...\n")

# Check everything in /kaggle/input
for dirname, _, filenames in os.walk('/kaggle/input'):
    if filenames:  # Only show directories with files
        print(f"📁 {dirname}")
        for filename in filenames:
            print(f"   ✓ {filename}")
        print()

🔍 Checking ALL Kaggle inputs...

📁 /kaggle/input/bitext
   ✓ Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11 2.csv



In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bitext/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11 2.csv


In [3]:
import pandas as pd
import numpy as np
import re
import random

from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [4]:
# Load the Bitext CSV using the correct path
DATA_PATH = "/kaggle/input/bitext/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11 2.csv"
df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
print(df.columns)
df.head()

Shape: (26872, 5)
Index(['flags', 'instruction', 'category', 'intent', 'response'], dtype='object')


,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [5]:
df["intent"].value_counts().head(20)

intent
contact_customer_service    1000
complaint                   1000
check_invoice               1000
switch_account              1000
edit_account                1000
contact_human_agent          999
check_payment_methods        999
delivery_period              999
newsletter_subscription      999
get_invoice                  999
payment_issue                999
registration_problems        999
cancel_order                 998
place_order                  998
track_refund                 998
change_order                 997
set_up_shipping_address      997
check_refund_policy          997
create_account               997
get_refund                   997
Name: count, dtype: int64

In [6]:
# Detect placeholders
import re

# pattern {{...}} as given in assignment
placeholder_pattern = re.compile(r"\{\{([^}]+)\}\}")

def find_placeholders(text):
    return placeholder_pattern.findall(str(text))

df["placeholders"] = df["instruction"].apply(find_placeholders)
df["num_placeholders"] = df["placeholders"].apply(len)

# how many rows have 0,1,2... placeholders
df["num_placeholders"].value_counts().head()

num_placeholders
0    20202
1     6298
2      372
Name: count, dtype: int64

In [7]:
# inspect some rows that have placeholders:
df[df["num_placeholders"] > 0][["instruction", "placeholders"]].head(10)

,instruction,placeholders
0,question about cancelling order {{Order Number}},[Order Number]
1,i have a question about cancelling oorder {{Or...,[Order Number]
2,i need help cancelling puchase {{Order Number}},[Order Number]
3,I need to cancel purchase {{Order Number}},[Order Number]
4,"I cannot afford this order, cancel purchase {{...",[Order Number]
5,can you help me cancel order {{Order Number}}?,[Order Number]
6,"I can no longer afford order {{Order Number}},...",[Order Number]
7,I am trying to cancel purchase {{Order Number}},[Order Number]
8,I have got to cancel purchase {{Order Number}},[Order Number]
9,i need help canceling purchase {{Order Number}},[Order Number]


In [8]:
# Before we write the replacement function, let’s quickly check all unique placeholder names so we know what to handle.

# collect all unique placeholder keys
all_placeholders = sorted({ph for row in df["placeholders"] for ph in row})
all_placeholders, len(all_placeholders)

(['Account Category',
  'Account Type',
  'Currency Symbol',
  'Delivery City',
  'Delivery Country',
  'Invoice Number',
  'Order Number',
  'Person Name',
  'Refund Amount'],
 9)

In [9]:
# Define synthetic generators for these 9 placeholders

# Synthetic value generators tailored to our 9 placeholder types

def random_order_number():
    # e.g. ORD-2025-38493
    return f"ORD-2025-{random.randint(10000, 99999)}"

def random_invoice_number():
    # e.g. INV-483920
    return f"INV-{random.randint(100000, 999999)}"

def random_person_name():
    names = [
        "Maria Garcia", "Alex Smith", "Li Wei", "Aisha Khan",
        "David Johnson", "Sofia Martinez", "James Lee", "Emily Chen"
    ]
    return random.choice(names)

def random_account_category():
    categories = ["Personal", "Business", "Premium", "Student"]
    return random.choice(categories)

def random_account_type():
    types = ["Checking", "Savings", "Credit Card", "Loan"]
    return random.choice(types)

def random_currency_symbol():
    symbols = ["$", "€", "£", "¥"]
    return random.choice(symbols)

def random_delivery_city():
    cities = ["New York", "Toronto", "London", "Sydney", "Berlin"]
    return random.choice(cities)

def random_delivery_country():
    countries = ["United States", "Canada", "United Kingdom", "Australia", "Germany"]
    return random.choice(countries)

def random_refund_amount():
    # e.g. $15.99, $120.00, etc.
    amount = round(random.uniform(5, 300), 2)
    return f"${amount:.2f}"

In [10]:
# Define the replacement function
def replace_placeholders_in_text(text):
    """
    Replace all {{...}} placeholders in one instruction string
    with realistic synthetic values based on the placeholder key.
    """
    text = str(text)

    def _replacement(match):
        key_raw = match.group(1).strip()
        key = key_raw.lower()

        if "order number" in key:
            return random_order_number()
        if "invoice number" in key:
            return random_invoice_number()
        if "person name" in key:
            return random_person_name()
        if "account category" in key:
            return random_account_category()
        if "account type" in key:
            return random_account_type()
        if "currency symbol" in key:
            return random_currency_symbol()
        if "delivery city" in key:
            return random_delivery_city()
        if "delivery country" in key:
            return random_delivery_country()
        if "refund amount" in key:
            return random_refund_amount()

        # Fallback in case something unexpected appears
        return "INFO"

    return placeholder_pattern.sub(_replacement, text)

In [11]:
# Apply the replacement and inspect a sample

# Create cleaned instruction column
df["instruction_clean"] = df["instruction"].apply(replace_placeholders_in_text)

# Look at a random sample of rows before vs after
df[["instruction", "instruction_clean"]].sample(10, random_state=RANDOM_SEED)


,instruction,instruction_clean
9329,I can't talk with a human agent,I can't talk with a human agent
4160,I have got to locate hte bills from {{Person N...,I have got to locate hte bills from Li Wei
18500,"I cannot pay, help me to inform of a problem w...","I cannot pay, help me to inform of a problem w..."
8840,I want help speaking to customer service,I want help speaking to customer service
5098,I try to see th accepted payment options,I try to see th accepted payment options
17250,where to sign up to the company nmewsletter,where to sign up to the company nmewsletter
3589,I'd like to see the withdrwaal fee how can i d...,I'd like to see the withdrwaal fee how can i d...
9043,I want to speak with someone,I want to speak with someone
15800,can you help me getting bill #85632?,can you help me getting bill #85632?
4384,I don't know how to take a quick look at invoi...,I don't know how to take a quick look at invoi...


**Observation: Based on the samples generated from the last step, we can tell that rows without placeholders stayed the same. For example, the row with {{Person Name}} became “Li Wei” in instruction_clean.**

In [12]:
# This step to make sure no placeholders remain. 

# Count remaining placeholders using the same regex
df["num_placeholders_after"] = df["instruction_clean"].apply(
    lambda x: len(find_placeholders(x))
)
df["num_placeholders_after"].value_counts()

num_placeholders_after
0    26872
Name: count, dtype: int64

In [13]:
# An extra sanity check: 
df["instruction_clean"].str.contains("{{", regex=False).sum()

0

**num_placeholders_after shows 0 for all 26,872 rows.
The str.contains("{{") check also returns 0.
Thus, we have confirmed that every placeholder has been removed successfully.** 

In [14]:
# Build df_clean and save bitext_cleaned.csv

OUTPUT_DIR = "/kaggle/working"

# Keep only the cleaned instruction and the label
df_clean = df[["instruction_clean", "intent"]].rename(
    columns={"instruction_clean": "instruction"}
)

print("Cleaned shape:", df_clean.shape)
df_clean.head()

# Save the full cleaned dataset
df_clean.to_csv(f"{OUTPUT_DIR}/bitext_cleaned.csv", index=False)

Cleaned shape: (26872, 2)


In [15]:
# Create the splits in memory

# Prepare features and labels from the cleaned dataset
X = df_clean["instruction"]
y = df_clean["intent"]

# 1) First split: reserve 10% as test set
X_temp, X_test, y_temp, y_test = train_test_split(
    X,
    y,
    test_size=0.10,
    stratify=y,
    random_state=RANDOM_SEED
)

# 2) Second split: from the remaining 90%, take ~11.11% as val
#    so that val is ~10% of the original total
X_train, X_val, y_train, y_val = train_test_split(
    X_temp,
    y_temp,
    test_size=0.1111,
    stratify=y_temp,
    random_state=RANDOM_SEED
)

len(X_train), len(X_val), len(X_test)

(21497, 2687, 2688)

In [16]:
# Save train.csv, val.csv, test.csv

def save_split(name, X_part, y_part):
    split_df = pd.DataFrame({
        "instruction": X_part,
        "intent": y_part
    })
    split_df.to_csv(f"{OUTPUT_DIR}/{name}.csv", index=False)
    print(name, "shape:", split_df.shape)

save_split("train", X_train, y_train)
save_split("val",   X_val,   y_val)
save_split("test",  X_test,  y_test)

train shape: (21497, 2)
val shape: (2687, 2)
test shape: (2688, 2)


In [17]:
import os
import pandas as pd

print("🔍 Checking for output files...\n")

output_dir = "/kaggle/working"
files = os.listdir(output_dir)

print(f"Files in {output_dir}:")
print("="*60)

if files:
    for f in sorted(files):
        if f.endswith('.csv'):
            file_path = os.path.join(output_dir, f)
            size = os.path.getsize(file_path) / (1024 * 1024)  # MB
            
            # Count rows
            df = pd.read_csv(file_path)
            rows = len(df)
            
            print(f"✅ {f:30s} | {size:.2f} MB | {rows:,} rows")
    
    print("\n✅ Files exist! Use 'Save Version' to download them.")
else:
    print("❌ No files found!")

🔍 Checking for output files...

Files in /kaggle/working:
✅ bitext_cleaned.csv             | 1.61 MB | 26,872 rows
✅ test.csv                       | 0.16 MB | 2,688 rows
✅ train.csv                      | 1.29 MB | 21,497 rows
✅ val.csv                        | 0.16 MB | 2,687 rows

✅ Files exist! Use 'Save Version' to download them.
